In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)
model = model.cuda()

/home/omar-ayyub/p/power_steering/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


In [17]:
print(model.model.layers[7].mlp.up_proj)

Linear(in_features=1024, out_features=3072, bias=False)


In [3]:
prompt = "i am the best"
inputs = tokenizer(prompt, return_tensors='pt').to('cuda')

with torch.no_grad():
    outputs = model(inputs["input_ids"], output_hidden_states=True)

In [4]:
print(len(outputs.hidden_states))
print(outputs.hidden_states[0].shape)

29
torch.Size([1, 4, 1024])


In [ ]:
source_idx = 7
target_idx = 20

down_proj = model.model.layers[source_idx].mlp.down_proj

down_proj.bias = torch.nn.Parameter(torch.zeros(1024, device='cuda'))

In [ ]:
steering_vec = torch.randn(1024, device='cuda')
captured_target = {}

def steering_hook(module, input, output):
    return output + steering_vec

def capture_hook(module, input, output):
    captured_target['value'] = output

handle1 = down_proj.register_forward_hook(steering_hook)

target_layer = model.model.layers[target_idx]
handle2 = target_layer.register_forward_hook(capture_hook)

steer_zero = torch.zeros(1024, device='cuda')
with torch.no_grad():
    _ = model(inputs['input_ids'])

print(captured_target['value'].shape)

torch.Size([1, 4, 1024])


In [28]:
# First, capture the unsteered target
steering_vec_data = torch.zeros(1024, device="cuda")

def steering_hook(module, input, output):
    return output + steering_vec_data

# Re-register with updated function (remove old one first)
handle1.remove()
handle1 = down_proj.register_forward_hook(steering_hook)

# Get unsteered baseline
with torch.no_grad():
    _ = model(inputs["input_ids"])
    unsteered_target = captured_target["value"].clone()

# Now steer with a random vector
steering_vec_data = torch.randn(1024, device="cuda")
steering_vec_data = steering_vec_data / steering_vec_data.norm()  # normalize
steering_vec_data = steering_vec_data * 10  # scale it up

with torch.no_grad():
    _ = model(inputs["input_ids"])
    steered_target = captured_target["value"].clone()

diff = (steered_target - unsteered_target).norm()
print(f"Displacement: {diff.item()}")

Displacement: 62.04249954223633


In [29]:
steering_vec_data = torch.zeros(1024, device='cuda', requires_grad=True)

handle1.remove()

def steering_hook(module, input, output):
    return output + steering_vec_data

handle1 = down_proj.register_forward_hook(steering_hook)

_ = model(inputs['input_ids'])
target = captured_target['value']

u = torch.randn_like(target)

grad = torch.autograd.grad(target, steering_vec_data, grad_outputs=u)[0]

print(grad.shape)

torch.Size([1024])


/home/omar-ayyub/p/power_steering/.venv/lib/python3.12/site-packages/torch/autograd/graph.py:841: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:270.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
